In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

In [2]:
data_in = np.loadtxt(fname='./test-score.csv', delimiter=',')
x_train = torch.from_numpy(data_in[:, 0:3]).float()             # (N, 3)
y_train = torch.from_numpy(data_in[:, 3]).unsqueeze(1).float()  # (N, 1)

In [3]:
class LinearRegressionModel(nn.Module):
    #__init__(): 객체가 갖는 속성값을 초기화하는 역할 -> 객체가 생성될 때 자동으로 호출
    # super()호출 >>>> 오버라이딩,  nn.Module 클래스의 속성들을 가지고 초기화 
    def __init__(self, W, b):
        super().__init__()
        self.linear = nn.Linear(3, 1)
        self.linear.bias = nn.Parameter(b, requires_grad=True)
        self.linear.weight = nn.Parameter(W.T, requires_grad=True)

    #forward() :모델이 학습데이터를 입력받아서 forward 연산을 진행 -> 객체를 데이터와 함께 호출하면 자동으로 실행
    # forward 연산: H(x)  식에 입력 x로부터 예측된 y를 얻는 것
    
    def forward(self, x):
        return self.linear(x)

In [4]:
W = torch.zeros((3, 1))
b = torch.zeros(1)
# model.parameters([W, b])
model = LinearRegressionModel(W, b)
# model.parameters()


In [5]:
# Set up the optimizer
optimizer = optim.SGD(model.parameters(), lr=1e-5)
nb_epochs = 1000

In [6]:
model.train()

for epoch in range(nb_epochs + 1):
    # Compute H(x)
    prediction = model(x_train)

    # Compute the cost with MSE
    cost = F.mse_loss(prediction, y_train)
    # Update H(x)
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    # print epoch and cost for every 20 epochs
    
    if epoch % 100 == 0:
        params = list(model.parameters())
        W = params[0].detach().numpy()
        b = params[1].item()
        print('Epoch {:4d}/{} W: {}, b: {:.3f} Cost: {:.6f}'.format(
            epoch, nb_epochs, tuple(*W), b, cost.item()
        ))



Epoch    0/1000 W: (0.26151597, 0.26294398, 0.26934156), b: 0.003 Cost: 26811.960938
Epoch  100/1000 W: (0.65932953, 0.66196424, 0.7035563), b: 0.008 Cost: 11.335224
Epoch  200/1000 W: (0.6515235, 0.653385, 0.7195268), b: 0.008 Cost: 10.945315
Epoch  300/1000 W: (0.64392394, 0.64524937, 0.7348633), b: 0.008 Cost: 10.585911
Epoch  400/1000 W: (0.63652545, 0.63753605, 0.7495918), b: 0.008 Cost: 10.254474
Epoch  500/1000 W: (0.6293219, 0.63022506, 0.7637377), b: 0.008 Cost: 9.948803
Epoch  600/1000 W: (0.62230855, 0.62329745, 0.77732414), b: 0.008 Cost: 9.666830
Epoch  700/1000 W: (0.6154797, 0.616735, 0.7903745), b: 0.008 Cost: 9.406623
Epoch  800/1000 W: (0.60883063, 0.61052006, 0.8029105), b: 0.008 Cost: 9.166456
Epoch  900/1000 W: (0.60235626, 0.6046361, 0.81495345), b: 0.008 Cost: 8.944720
Epoch 1000/1000 W: (0.5960518, 0.59906715, 0.8265234), b: 0.008 Cost: 8.739944


In [7]:
model.eval()
# Put the test input
new_var = torch.FloatTensor([[73, 80, 75]])
# Get the prediction value.
pred_y = model(new_var)
print(pred_y)


tensor([[153.4341]], grad_fn=<AddmmBackward0>)
